# Create Public School Machine Learning Datasets
** This program creates all the _ML datasets in the NCEA repository.** 
* This notebook reads each School Dataset file located at \EducationDataNC\ *schoolYear* \School Datasets\ as input data.
* Different school years are processed by changing the *schoolYear* parameter.
* Different input / output files are processed / created by changing the *inputFileName* paramter in the cell below.  
* While a single program is used to create all the _ML datasets, one program copy per dataset is maintained in the repositiory so the dataset specific tranformation reports may be reviewed. 

**Datasets ending in ML are preprocessed for Machine Learning and go through the following transformations: **
1. Missing student body racial compositions are imputed using district averages.
2. Columns that have the same value in every single row are deleted.
3. Columns that have a unique value in every single row (all values are different) are deleted.
4. Empty columns (all values are NA or NULL) are deleted.
5. Numeric columns with more than the percentage of missing values specified by the *missingThreshold* parameter.
6. Remaining numeric, non-race columns with missing values are imputed / populated with 0.  In many cases, schools are not reporting values when they are zero. However, mean imputation or some other more sophisticated strategy might be considered here.
7. Categorical / text based columns with > *uniqueThreshold* unique values are deleted.
8. All remaining categorical / text based columns are one-hot encoded.  In categorical columns, one-hot encoding creates one new boolean / binary field per unique value in the target column, converting all categorical columns to a numeric data type. 
9. Duplicated or highly similar columns with > 95% correlation are delelted.    

In [22]:
#import required Libraries
import pandas as pd
import numpy as np
import os
import string

#**********************************************************************************
# Set the following variables before running this code!!!
#**********************************************************************************
#All raw data files are processed for the year below
schoolYear = 2019

#Location where copies of the raw data files will be read in from csv files.
dataDir = 'C:/Users/Jake/Documents/GitHub/EducationDataNC/' + str(schoolYear) + '/School Datasets/'

#Name of the file to be processed
#inputFileName = 'PublicSchools2019'
inputFileName = 'PublicHighSchools2019'
#inputFileName = 'PublicMiddleSchools2019'
#inputFileName = 'PublicElementarySchools2019'

#Input file being transformed for machine learning 
inputFile = dataDir + inputFileName + '.csv'

#Location where the new school datasets will be created.
outputDir = 'C:/Users/Jake/Documents/GitHub/EducationDataNC/' + str(schoolYear) + '/Machine Learning Datasets/'

#Missing Data Threshold (Per Column)
missingThreshold = 0.60

#Unique Value Threshold (Per Column)
#Delete Columns >  uniqueThreshold unique values prior to one-hot encoding. 
#(each unique value becomes a new column during one-hot encoding)
uniqueThreshold = 200

#Read in the School Data File
schoolData = pd.read_csv(inputFile, low_memory=False, dtype={'agency_code': object})
print('*********Start: Beginning Column and Row Counts********************************************')
schoolData.info(verbose=False)

#Select only public schools as charter schools are missing data for many columns.
schoolData = schoolData[(schoolData['designation_type'] == 'P')] #& (schoolData['avg_student_num'] > 0)] -rcd_admin missing

print('\r\n*********After: Selecting Only Public School Campuses**********************************')
schoolData.info(verbose=False)

#Save primary key
agency_code = schoolData['agency_code']
#Convert zip code to string
schoolData['zip'] = schoolData['zip'].astype('object')

*********Start: Beginning Column and Row Counts********************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685 entries, 0 to 684
Columns: 1485 entries, agency_code to welcome_url
dtypes: float64(1435), int64(1), object(49)
memory usage: 7.8+ MB

*********After: Selecting Only Public School Campuses**********************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 613 entries, 2 to 684
Columns: 1485 entries, agency_code to welcome_url
dtypes: float64(1435), int64(1), object(49)
memory usage: 6.9+ MB


# Prepare Consolidated Dataset for Machine Learning
**Below we perform operations on the entire dataset to remove columns and update row values that could cause problems during machine learning.**

## Student Body Racial Composition Features 
**Impute / update missing Student Body Racial Composition Fields using mean imputation.**
* When there are no racial composition percentages for a particular school campus / agency_code, fill in the missing values 

In [23]:
#Get Student Body Racial Composition Fields
raceCompositionFields = schoolData.filter(regex='Indian|Asian|Hispanic|Black|White|PacificIsland|TwoOrMore|Minority')\
                                  .filter(regex='Pct').columns
    
rowsBefore = schoolData[raceCompositionFields].isnull().T.any().T.sum()

#Update missing race values with the district average when avaiable (No district averages for charter schools) 
schoolData[raceCompositionFields] = schoolData.groupby('lea_code')[raceCompositionFields]\
                                              .transform(lambda x: x.fillna(x.mean()))

    #Review dataset contents after Racial Composition Imputation
print('*********After: Updating Missing Racial Compostion Values****************************')   
rowsAfter = schoolData[raceCompositionFields].isnull().T.any().T.sum()
rowsUpdated = rowsBefore - rowsAfter
print('Rows Updated / Imputed: ', rowsUpdated) 
print('\r\nTotal Rows Missing Racial Compositions By District Code') 
schoolData['lea_code'][schoolData[raceCompositionFields].isnull().T.any().T].value_counts()

*********After: Updating Missing Racial Compostion Values****************************
Rows Updated / Imputed:  9

Total Rows Missing Racial Compositions By District Code


010LEA    9
070LEA    5
060LEA    4
040LEA    3
090LEA    3
080LEA    2
050LEA    2
020LEA    2
030LEA    1
Name: lea_code, dtype: int64

## Remove Columns with Problematic Data
**Here we remove entire columns that could cause problems during machine learning.  The following operations are performed:**
* Remove any columns that have the same value in every single row.
* Remove any columns that have a unique value in every single row (all values are different).
* Remove empty columns (all values are NA or NULL).

In [24]:
#Remove any fields that have the same value in all rows
UniqueValueCounts = schoolData.nunique(dropna=False)
SingleValueCols = UniqueValueCounts[UniqueValueCounts == 1].index
schoolData = schoolData.drop(SingleValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing columns with the same value in every row.*******************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(SingleValueCols))

*********After: Removing columns with the same value in every row.*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 613 entries, 2 to 684
Columns: 731 entries, agency_code to welcome_url
dtypes: float64(687), object(44)
memory usage: 3.4+ MB

Columns Deleted:  754


In [25]:
#Remove any fields that have unique values in every row
schoolDataRecordCt = schoolData.shape[0]
UniqueValueCounts = schoolData.apply(pd.Series.nunique)
AllUniqueValueCols = UniqueValueCounts[UniqueValueCounts == schoolDataRecordCt].index
schoolData = schoolData.drop(AllUniqueValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing columns with unique values in every row.*******************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(AllUniqueValueCols))

*********After: Removing columns with unique values in every row.*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 613 entries, 2 to 684
Columns: 731 entries, agency_code to welcome_url
dtypes: float64(687), object(44)
memory usage: 3.4+ MB

Columns Deleted:  0


In [26]:
#Remove any empty fields (null values in every row)
schoolDataRecordCt = schoolData.shape[0]
NullValueCounts = schoolData.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts == schoolDataRecordCt].index
schoolData = schoolData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('*********After: Removing columns with null / blank values in every row.*************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols))

*********After: Removing columns with null / blank values in every row.*************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 613 entries, 2 to 684
Columns: 731 entries, agency_code to welcome_url
dtypes: float64(687), object(44)
memory usage: 3.4+ MB

Columns Deleted:  0


## Handle Other Missing Values Types
* Here we eliminate any numeric columns with more than the percentage of missing values specified by the *missingThreshold* parameter.
* All remaining non-race, numeric column missing values are populated with 0.
* In many cases, it seems that schools are not simply not reporting values when they are zero. However, mean imputation or some other strategy might be considered.

In [27]:
#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
sD_boolean = schoolData.loc[:, (schoolData.dtypes == bool) ]
sD_nominal = schoolData.loc[:, (schoolData.dtypes == object)]
sD_continuous = schoolData.loc[:, (schoolData.dtypes != bool) & (schoolData.dtypes != object)]
print ("Boolean Columns: ", sD_boolean.shape[1])
print ("Nominal Columns: ", sD_nominal.shape[1])
print ("Continuous Columns: ", sD_continuous.shape[1])
print ("Columns Accounted for: ", sD_nominal.shape[1] + sD_continuous.shape[1] + sD_boolean.shape[1])

Boolean Columns:  0
Nominal Columns:  44
Continuous Columns:  687
Columns Accounted for:  731


In [28]:
#Eliminate continuous columns with more than missingThreshold percentage of missing values
schoolDataRecordCt = sD_continuous.shape[0]
missingValueLimit = schoolDataRecordCt * missingThreshold
NullValueCounts = sD_continuous.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts >= missingValueLimit].index
schoolData = schoolData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('*********After: Removing columns with >= missingThreshold % of missing values******')
schoolData.info(verbose=False)
print('\r\nColumns Deleted: ', len(NullValueCols))

*********After: Removing columns with >= missingThreshold % of missing values******
<class 'pandas.core.frame.DataFrame'>
Int64Index: 613 entries, 2 to 684
Columns: 415 entries, agency_code to welcome_url
dtypes: float64(371), object(44)
memory usage: 1.9+ MB

Columns Deleted:  316


## One-Hot Encoding of Categorical Variables
**All categorical / string variables are converted to numberic variables via one hot encoding.  Each unique row value will become a new binary / numeric column in the dataset.**
* All remaining categorical columns are one-hot encoded.  
* In categorical columns, one-hot encoding creates one new boolean / binary field per unique value in the target column, converting all categorical columns to a numeric data type. 
* Prior to one-hot encoding, columns with > *uniqueThreshold* unique values are deleted.  

In [29]:
#Delete categorical columns with > uniqueThreshold unique values 
#(Each unique value becomes a column during one-hot encoding)
oneHotUniqueValueCounts = schoolData[sD_nominal.columns].apply(lambda x: x.nunique())
oneHotUniqueValueCols = oneHotUniqueValueCounts[oneHotUniqueValueCounts >= uniqueThreshold].index
schoolData.drop(oneHotUniqueValueCols, axis=1, inplace=True) 

#Remove categorical columns which do not add value to machine learning
#Grades columns dropped since they all have indicator fields (device_access, device_home, BYOD)
#with less unique values to one-hot encode
schoolData.drop(['welcome_url','device_access_grades',
                 'device_home_grades','byod_grades'], axis=1, inplace=True) 

#Review dataset contents one hot high unique value drops
print('*********After: Removing columns with >= uniqueThreshold unique values***********')
schoolData.info(verbose=False)
print('\r\nColumns Deleted: ', len(oneHotUniqueValueCols))
print(oneHotUniqueValueCols)
print('Dropped Manually: welcome_url,device_access_grades,device_home_grades,byod_grades')

*********After: Removing columns with >= uniqueThreshold unique values***********
<class 'pandas.core.frame.DataFrame'>
Int64Index: 613 entries, 2 to 684
Columns: 402 entries, category_code to pct_sat_participation
dtypes: float64(371), object(31)
memory usage: 1.9+ MB

Columns Deleted:  9
Index(['agency_code', 'name', 'street_addr', 'city', 'zip', 'phone', 'url',
       'School Name', 'Book_TitlesPer_Student'],
      dtype='object')
Dropped Manually: welcome_url,device_access_grades,device_home_grades,byod_grades


In [30]:
#Isolate remaining categorical variables
begColumnCt = len(schoolData.columns)
sD_nominal = schoolData.loc[:, (schoolData.dtypes == object)]

#one hot encode categorical variables
schoolData = pd.get_dummies(data=schoolData, 
                       columns=sD_nominal.columns, drop_first=True)

#Determine change in column count
endColumnCt = len(schoolData.columns)
columnsAdded = endColumnCt - begColumnCt

#Review dataset contents one hot high unique value drops
print('Columns To One-Hot Encode: ', len(sD_nominal.columns))
print('\r\n*********After: Adding New Columns Via One-Hot Encoding*************************')
schoolData.info(verbose=False)
print('\r\nNew Columns Created Via One-Hot Encoding: ', columnsAdded)

Columns To One-Hot Encode:  31

*********After: Adding New Columns Via One-Hot Encoding*************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 613 entries, 2 to 684
Columns: 913 entries, IndianPct to devices_7.25
dtypes: float64(371), uint8(542)
memory usage: 2.1 MB

New Columns Created Via One-Hot Encoding:  511


In [31]:
#Remove any fields that have the same value in all rows after one-hot encoding
UniqueValueCounts = schoolData.nunique(dropna=False)
SingleValueCols = UniqueValueCounts[UniqueValueCounts == 1].index
schoolData = schoolData.drop(SingleValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing one-hot encoded columns with the same value in every row.*******************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(SingleValueCols))

*********After: Removing one-hot encoded columns with the same value in every row.*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 613 entries, 2 to 684
Columns: 913 entries, IndianPct to devices_7.25
dtypes: float64(371), uint8(542)
memory usage: 2.1 MB

Columns Deleted:  0


## Impute any Remaining Missing Values as Zero

In [32]:
#Print out all the missing value rows
pd.set_option('display.max_rows', 1000)

print('\r\n*********The Remaining Missing Values Below will be set to Zero!*************************')

#Check for Missing values 
missing_values = schoolData.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
missing_values




*********The Remaining Missing Values Below will be set to Zero!*************************


,Variable Name,Number Missing Values
0,IndianPct,31
1,AsianPct,31
2,HispanicPct,31
3,BlackPct,31
4,WhitePct,31
5,PacificIslandPct,31
6,TwoOrMorePct,31
7,MinorityPct,31
8,IndianMalePct,31
9,AsianMalePct,31


In [33]:
#Replace all remaining NaN with 0
schoolData = schoolData.fillna(0)

#Check for Missing values after final imputation 
missing_values = schoolData.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
missing_values

,Variable Name,Number Missing Values


## Identify and Remove Highly Correlated Features
**Find and remove any columns / features that are > 95% correlated**
* https://stackoverflow.com/questions/39409866/correlation-heatmap
* https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
* https://codeyarns.com/2015/04/20/how-to-change-font-size-in-seaborn/

In [34]:
# calculate the correlation matrix
corr_matrix  = schoolData.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [35]:
#Get all of the correlation values > 95%
x = np.where(upper > 0.95)

#Display all field combinations with > 95% correlation
cf = pd.DataFrame()
cf['Field1'] = upper.columns[x[1]]
cf['Field2'] = upper.index[x[0]]

#Get the correlation values for every field combination. (There must be a more pythonic way to do this!)
corr = [0] * len(cf)
for i in range(0, len(cf)):
    corr[i] =  upper[cf['Field1'][i]][cf['Field2'][i]] 
    
cf['Correlation'] = corr

print ('There are ', str(len(cf['Field1'])), ' field correlations > 95%.')
cf

There are  249  field correlations > 95%.


,Field1,Field2,Correlation
0,IndianMalePct,IndianPct,0.981089
1,IndianFemalePct,IndianPct,0.979129
2,pct_ACALL_FEM_ACT,pct_ACALL_ALL_ACT,0.977490
3,pct_ACALL_MALE_ACT,pct_ACALL_ALL_ACT,0.971186
4,pct_ACALL_NEDS_ACT,pct_ACALL_ALL_ACT,0.975917
5,pct_ACALL_NSWD_ACT,pct_ACALL_ALL_ACT,0.998143
6,pct_ACEN_ALL_ACT,pct_ACALL_ALL_ACT,0.973010
7,pct_ACEN_NSWD_ACT,pct_ACALL_ALL_ACT,0.968123
8,pct_ACMA_ALL_ACT,pct_ACALL_ALL_ACT,0.959608
9,pct_ACMA_NSWD_ACT,pct_ACALL_ALL_ACT,0.958292


In [36]:
#The old way
#print('Dropping the following ', str(len(to_drop)), ' highly correlated fields.')
#to_drop

# Comparing highly correlated unique field names in field 1 vs field 2 list above will allow you to 
# remove the least amount of fields possible.  
#len(cf['Field1'].unique())
#len(cf['Field2'].unique())

print('Dropping the following ', str(len(cf['Field2'].unique())), ' highly correlated, unique fields.')
cf['Field2'].unique()

Dropping the following  177  highly correlated, unique fields.


array(['IndianPct', 'pct_ACALL_ALL_ACT', 'pct_ACALL_FEM_ACT',
       'pct_ACALL_MALE_ACT', 'pct_ACALL_NEDS_ACT', 'pct_ACALL_NSWD_ACT',
       'pct_ACCO_AIG_ACT', 'pct_ACCO_ALL_ACT', 'pct_ACCO_FEM_ACT',
       'pct_ACCO_MALE_ACT', 'pct_ACCO_NEDS_ACT', 'pct_ACCO_NSWD_ACT',
       'pct_ACCO_WH7_ACT', 'pct_ACEN_AIG_ACT', 'pct_ACEN_ALL_ACT',
       'pct_ACEN_FEM_ACT', 'pct_ACEN_MALE_ACT', 'pct_ACEN_NEDS_ACT',
       'pct_ACEN_NSWD_ACT', 'pct_ACEN_WH7_ACT', 'pct_ACMA_AIG_ACT',
       'pct_ACMA_ALL_ACT', 'pct_ACMA_FEM_ACT', 'pct_ACMA_MALE_ACT',
       'pct_ACMA_NEDS_ACT', 'pct_ACMA_NSWD_ACT', 'pct_ACMA_WH7_ACT',
       'pct_ACRD_AIG_ACT', 'pct_ACRD_ALL_ACT', 'pct_ACRD_FEM_ACT',
       'pct_ACRD_MALE_ACT', 'pct_ACRD_NEDS_ACT', 'pct_ACRD_NSWD_ACT',
       'pct_ACRD_WH7_ACT', 'pct_ACSC_ALL_ACT', 'pct_ACSC_NEDS_ACT',
       'pct_ACWR_ALL_ACT', 'pct_ACWR_FEM_ACT', 'pct_ACWR_NEDS_ACT',
       'pct_AIG_AWA', 'pct_ALL_AWA', 'pct_EXT_ALL_CGR',
       'pct_EXT_NEDS_CGR', 'pct_EXT_NSWD_CGR', 'pct_STD_AL

In [37]:
#Check columns before drop 
print('\r\n*********Before: Dropping Highly Correlated Fields*************************************')
schoolData.info(verbose=False)

# Drop the highly correlated features from our training data 
#schoolData = schoolData.drop(to_drop, axis=1)
schoolData = schoolData.drop(cf['Field2'], axis=1)

#Check columns after drop 
print('\r\n*********After: Dropping Highly Correlated Fields**************************************')
schoolData.info(verbose=False)


*********Before: Dropping Highly Correlated Fields*************************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 613 entries, 2 to 684
Columns: 913 entries, IndianPct to devices_7.25
dtypes: float64(371), uint8(542)
memory usage: 2.1 MB

*********After: Dropping Highly Correlated Fields**************************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 613 entries, 2 to 684
Columns: 736 entries, AsianPct to devices_7.25
dtypes: float64(292), uint8(444)
memory usage: 1.6 MB


In [38]:
#Restore the unit_code before saving
schoolData['agency_code'] = agency_code
#Save the final dataset to a .csv file
schoolData.to_csv(outputDir + inputFileName + '_ML.csv', sep=',', index=False)

In [39]:
print('*********FINAL DATASET DETAILS*********************************************************\r\n')
schoolData.info(verbose=True)

*********FINAL DATASET DETAILS*********************************************************

<class 'pandas.core.frame.DataFrame'>
Int64Index: 613 entries, 2 to 684
Data columns (total 737 columns):
AsianPct                                                                                                            float64
HispanicPct                                                                                                         float64
BlackPct                                                                                                            float64
WhitePct                                                                                                            float64
PacificIslandPct                                                                                                    float64
TwoOrMorePct                                                                                                        float64
MinorityPct                                                  

In [40]:
import sklearn
import pandas as pd

print('Sklearn Version: ' + sklearn.__version__)
print('Pandas Version: ' + pd.__version__)

Sklearn Version: 0.21.2
Pandas Version: 0.23.4


In [41]:
print('Output File Location:\r\n\r\n' + outputDir + inputFileName + '_ML.csv')

Output File Location:

C:/Users/Jake/Documents/GitHub/EducationDataNC/2019/Machine Learning Datasets/PublicHighSchools2019_ML.csv
